In [4]:
import pandas as pd

df = pd.read_csv('df_combined.csv',index_col=0)
df

,ID,技術指標_月RSI(10),技術指標_週RSI(10),技術指標_季K(9),個股19天乖離率,技術指標_乖離率(250日),技術指標_月ADX(14),技術指標_RSI(10),技術指標_週RSI(5),技術指標_季D(9),技術指標_月RSI(5),飆股
0,TR-79780,0.4669,0.5827,0.972400,0.4268,0.174500,0.9963,0.0019,0.8878,0.599600,0.3863,0
1,TR-191682,3.0209,2.6634,3.603000,1.3062,1.388400,2.5840,3.0428,2.8966,3.611700,2.3930,1
2,TR-147296,2.5408,2.6841,2.786500,2.2480,2.042800,1.5357,2.7986,2.8236,2.612400,2.5284,1
3,TR-124405,1.4456,0.3502,2.895800,1.0960,0.910300,2.1700,0.7997,0.2882,3.636200,0.5902,1
4,TR-32426,2.8228,1.8385,3.213200,1.1828,1.393000,2.1382,1.4787,1.6563,3.046900,2.0322,0
...,...,...,...,...,...,...,...,...,...,...,...,...
251075,TR-7890,1.9160,2.9218,1.600500,4.5632,2.744600,1.2116,3.7775,3.1119,1.109500,2.1367,0
251076,TR-130281,1.7068,2.0029,1.796893,1.8648,2.216066,0.5384,2.4217,2.0028,1.583484,1.7029,0
251077,TR-191682,3.0209,2.6634,3.603000,1.3062,1.388400,2.5840,3.0428,2.8966,3.611700,2.3930,1
251078,TR-17084,0.7540,1.5660,0.397900,1.2719,0.776100,0.4868,2.1629,2.2154,0.478400,0.9364,0


In [5]:
import catboost
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

x_col = df.drop(['ID','飆股'],axis=1).columns.tolist()
y_col = '飆股'

X_train, X_test, y_train, y_test = train_test_split(
    df[x_col], df[y_col], 
    test_size=0.2, random_state=42
)

# 3. 建立 Pool（可選）
train_pool = Pool(data=X_train, label=y_train)
test_pool = Pool(data=X_test, label=y_test)

In [6]:
# 4. 設定模型參數
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    eval_metric='F1',
    task_type='GPU',        # ✅ 使用 GPU
    devices='0',            # 使用第0張GPU
    verbose=1
)

In [7]:
# 5. 訓練模型
model.fit(train_pool, eval_set=test_pool)

0:	learn: 0.9024521	test: 0.9007202	best: 0.9007202 (0)	total: 24.7ms	remaining: 2.44s
1:	learn: 0.9059926	test: 0.9040428	best: 0.9040428 (1)	total: 37ms	remaining: 1.81s
2:	learn: 0.9091510	test: 0.9056516	best: 0.9056516 (2)	total: 52.1ms	remaining: 1.68s
3:	learn: 0.9104216	test: 0.9083388	best: 0.9083388 (3)	total: 63.7ms	remaining: 1.53s
4:	learn: 0.9170804	test: 0.9146177	best: 0.9146177 (4)	total: 72.6ms	remaining: 1.38s
5:	learn: 0.9167513	test: 0.9143123	best: 0.9146177 (4)	total: 80.9ms	remaining: 1.27s
6:	learn: 0.9182930	test: 0.9161238	best: 0.9161238 (6)	total: 89.8ms	remaining: 1.19s
7:	learn: 0.9209685	test: 0.9184965	best: 0.9184965 (7)	total: 97.6ms	remaining: 1.12s
8:	learn: 0.9195064	test: 0.9171763	best: 0.9184965 (7)	total: 105ms	remaining: 1.07s
9:	learn: 0.9226460	test: 0.9201185	best: 0.9201185 (9)	total: 113ms	remaining: 1.02s
10:	learn: 0.9249575	test: 0.9226334	best: 0.9226334 (10)	total: 121ms	remaining: 982ms
11:	learn: 0.9249855	test: 0.9226553	best: 0.9

In [8]:
pub_x = pd.read_csv('38_Public_Test_Set_and_Submmision_Template_V2/38_Public_Test_Set_and_Submmision_Template_V2/public_x.csv')
pub_x

,ID,外資券商_分點進出,外資券商_分點買賣力,外資券商_分點成交力(%),外資券商_分點吃貨比(%),外資券商_分點出貨比(%),外資券商_前1天分點進出,外資券商_前1天分點買賣力,外資券商_前1天分點成交力(%),外資券商_前1天分點吃貨比(%),...,上市加權指數前14天成交量,上市加權指數前15天成交量,上市加權指數前16天成交量,上市加權指數前17天成交量,上市加權指數前18天成交量,上市加權指數前19天成交量,上市加權指數前20天成交量,上市加權指數5天成交量波動度,上市加權指數10天成交量波動度,上市加權指數20天成交量波動度
0,PU-1,1.2219,NaN,1.2413,0.5313,0.5329,1.2258,NaN,1.2238,0.5228,...,0.3492,0.1014,0.4583,0.2233,0.5630,0.2876,0.2664,0.8630,0.4614,0.3086
1,PU-2,1.2219,NaN,1.2413,0.5313,0.5329,1.2243,1.2072,1.2238,0.5228,...,1.8991,2.3784,1.9220,2.3146,3.4425,2.8982,2.8924,1.6241,2.2758,1.8835
2,PU-3,3.2984,1.4356,1.8457,3.5096,2.8033,-2.3714,1.2767,1.3264,2.3178,...,0.6375,0.4582,0.8285,0.7927,1.4526,1.2864,1.1145,0.5992,2.3232,2.0103
3,PU-4,1.2211,-0.3363,0.7311,0.5313,1.0516,1.2258,NaN,1.2104,0.5228,...,0.5781,0.9577,0.6808,0.5511,0.1747,0.0290,0.1628,0.8125,0.7227,0.5577
4,PU-5,1.5689,1.5861,1.8969,2.4814,1.7553,1.0782,1.3010,1.2881,1.7097,...,1.0682,1.0271,0.9876,1.7390,1.3109,1.6806,1.3216,4.1865,2.7835,1.7238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25103,PU-25104,1.2219,NaN,1.2413,0.5313,0.5329,1.2250,1.2072,1.2238,0.5228,...,3.0928,2.4741,2.2177,2.4033,2.0818,2.5835,2.3398,1.3057,0.8594,1.4832
25104,PU-25105,1.3073,1.2751,1.3396,3.4861,3.2950,-0.0939,1.2131,1.2345,3.0997,...,3.3519,2.5249,2.8949,2.3010,2.3636,2.7702,2.3189,2.0434,1.9664,1.8610
25105,PU-25106,1.2041,-0.3363,0.3484,0.5313,1.4406,1.1811,1.0835,1.1827,0.5228,...,0.6593,0.9105,0.7159,0.6506,0.8340,1.0138,1.2428,0.3604,-0.2059,0.2469
25106,PU-25107,1.0892,0.6864,0.8777,0.8795,1.2398,1.4488,1.1700,1.2118,1.2927,...,1.0392,1.3660,1.2182,1.3742,1.0321,1.3339,1.9647,0.8373,0.9554,0.7367


In [9]:
y_pred = model.predict(pub_x[x_col])
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [10]:
submit = pd.read_csv('38_Public_Test_Set_and_Submmision_Template_V2/38_Public_Test_Set_and_Submmision_Template_V2/submission_template_public.csv')
submit['飆股'] = y_pred
submit.to_csv('submit.csv',index=False)